# 2.3 - Folium

https://python-visualization.github.io/folium/

In [ ]:
%pip install folium

In [ ]:
import folium

import pandas as pd

In [ ]:
# crear mapa

mapa=folium.Map(location=[19.41, -99.16], # lat-lon del centro del mapa
                tiles='openstreetmap',    # estilo del mapa
                zoom_start=12             # zoom inicial
               )

In [ ]:
# marcador

folium.CircleMarker([19.41, -99.16], # lat-lon del marcador
                    radius=10,       # tamaño del marcador, radio del circulo
                    icon=folium.Icon()).add_to(mapa)

In [ ]:
# guarda el mapa

mapa.save('images/mex.html')

In [ ]:
mapa

In [ ]:
# otro mapa

mapa2=folium.Map(location=[19.41, -99.16], 
                 tiles='stamentoner', 
                 zoom_start=12)

lat=[19.42, 19.4, 19.43, 19.39, 19.42, 19.38]
lng=[-99.1, -99.15, -99.13 ,-99.12, -99.09, -99.11]

for i in range(len(lat)):
    folium.Marker([lat[i], lng[i]], icon=folium.Icon(color='red')).add_to(mapa2)
    
    
mapa2.save('images/mex2.html')

mapa2

In [ ]:
import folium
from folium import plugins


m = folium.Map(
    location=[35.68159659061569, 139.76451516151428],
    zoom_start=16
)



# longitud y latitud por orden.
lines = [
    {
        'coordinates': [
            [139.76451516151428, 35.68159659061569],
            [139.75964426994324, 35.682590062684206],
        ],
        'dates': [
            '2017-06-02T00:00:00',
            '2017-06-02T00:10:00'
        ],
        'color': 'red'
    },
    {
        'coordinates': [
            [139.75964426994324, 35.682590062684206],
            [139.7575843334198, 35.679505030038506],
        ],
        'dates': [
            '2017-06-02T00:10:00',
            '2017-06-02T00:20:00'
        ],
        'color': 'blue'
    },
    {
        'coordinates': [
            [139.7575843334198, 35.679505030038506],
            [139.76337790489197, 35.678040905014065],
        ],
        'dates': [
            '2017-06-02T00:20:00',
            '2017-06-02T00:30:00'
        ],
        'color': 'green',
        'weight': 15,
    },
    {
        'coordinates': [
            [139.76337790489197, 35.678040905014065],
            [139.76451516151428, 35.68159659061569],
        ],
        'dates': [
            '2017-06-02T00:30:00',
            '2017-06-02T00:40:00'
        ],
        'color': '#FFFFFF',
    },
]




features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]







plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(m)

#display(m)



m.save('images/mapa_continuo.html')

m

In [ ]:
df=pd.read_json('../data/oficinas.json') 

df.head()

In [ ]:
# heatmap

mapa3=folium.Map([47.603122, -122.333253], zoom_start=4)

data=df[['lat', 'lng']].values

mapa3.add_child(plugins.HeatMap(data, radius=15))

mapa3

In [ ]:
# heatmap con movimiento

df=pd.read_csv('../data/test_folium.csv')

df.head()

In [ ]:
df.shape

In [ ]:
from folium.plugins import HeatMapWithTime as HMWT
from folium.plugins import HeatMap as HM

In [ ]:
# transformar segun datos de tiempo

df.pickup_datetime = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')

df['month'] = df.pickup_datetime.apply(lambda x: x.month)

df['week'] = df.pickup_datetime.apply(lambda x: x.week)

df['day'] = df.pickup_datetime.apply(lambda x: x.day)

df['hour'] = df.pickup_datetime.apply(lambda x: x.hour)

In [ ]:
df.head()

In [ ]:
# inicializa el mapa

def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=11):
    
    base_map = folium.Map(location=default_location, 
                          control_scale=True, 
                          zoom_start=default_zoom_start)
    
    return base_map

In [ ]:
mapa4=generateBaseMap()

In [ ]:
mapa4

In [ ]:
# se copia df y mes>4

df_copy = df[df.month>4].copy()
df_copy['count'] = 1

In [ ]:
# datos para heatmap

df_copy[['pickup_latitude', 
         'pickup_longitude', 
         'count']].groupby(['pickup_latitude', 
                            'pickup_longitude']).sum().sort_values('count', 
                                                                   ascending=False).head(10)

In [ ]:
base_map = generateBaseMap() # crea mapa

HM(data=df_copy[['pickup_latitude', 
                  'pickup_longitude', 
                  'count']].groupby(['pickup_latitude', 
                                     'pickup_longitude']).sum().reset_index().values.tolist(), 
    radius=8, 
        max_zoom=13).add_to(base_map) # pon los datos en heatmap

base_map.save('base_map.html')  # guarda

base_map   # representa

In [ ]:
df_hour_list = []
for hour in df_copy.hour.sort_values().unique():
    df_hour_list.append(df_copy.loc[df_copy.hour == hour, 
                                    ['pickup_latitude', 
                                     'pickup_longitude', 
                                     'count']].groupby(['pickup_latitude', 
                                                        'pickup_longitude']).sum().reset_index().values.tolist())

In [ ]:
base_map = generateBaseMap(default_zoom_start=11) # crea mapa

HMWT(df_hour_list, 
     radius=5, 
     gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, 
     min_opacity=0.5, 
     max_opacity=0.8, 
     use_local_extrema=True).add_to(base_map) # datos pata heatmap con tiempo


base_map  # representa

In [ ]:
base_map.save('images/mapa.html') # guarda

# UFO

In [ ]:
import pandas as pd
import folium
from folium import plugins

In [ ]:
# Import data
ufo=pd.read_csv('../data/ufo.csv', index_col=False)

ufo=ufo.drop(columns = 'Unnamed: 0')

ufo.head()

In [ ]:
data=ufo[(ufo.country=='us') & (ufo.year>=2004)]

data.head()

In [ ]:
mapa = folium.Map([36.174465, -86.767960], zoom_start=4)

value_data = data[['latitude', 'longitude']].values

mapa.add_child(plugins.HeatMap(value_data, radius=15))

mapa